In [524]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from pathlib import Path
data_location = Path("../Data/Tableau_Terra_workspace")

# Create the terra workspace 
Read all these weird files and try to merge them to the best of my capabilities:

1. Read and merge CLF with Premium
2. Read and merge Panel data
3. Read RNA data
4. Read and merge WES data - UPCOMMING
5. Apply loop through rows
6. Duplicate rows and add conditions
7. Check number of datapoints

Note, losing alot of information due to inconsistencies in the datasets. Another note this workflow eats up memory, because I keep meta data open, just something to note

### 1 Read and merge CLF with Premium

In [796]:
#CLF data and cleaning
CLF_data_meta = pd.read_csv(data_location / "CLF_Cohort_Summary_full_data.csv", encoding = 'utf-8', sep='\t')
CLF_data = CLF_data_meta[["Cell Line", "PT-ID", "SM-ID", "Status", "Resolution", "T/N", "Primary Disease", "Tissue site", "Tumor type", "CELL_LINE_ID(raw)", "Media type", "Growth Properties", "Flask coating", "Starting Material Type", "Fail Mode"]]
CLF_data = CLF_data.replace(r'^\s*$', np.nan, regex=True)
CLF_data = CLF_data.rename(columns={"Cell Line": "Cell Line Starting Sample", "Tissue site": "Tissue Site", "Tumor type": "Tumor Type", "CELL_LINE_ID(raw)":"Cell Line", "Growth Properties": "Dimension"})
CLF_data["Dataset"] = "CLF"

In [805]:
#Premium data and cleaning
Premium_data_meta = pd.read_csv(data_location / "Premium_Cohort_Summary_full_data.csv", encoding = 'utf-8', sep='\t')
Premium_data = Premium_data_meta[["Cell Line Starting Sample", "PT_ID", "Passage Sample", "JIRA Status", "Resolution", "T/N", "Primary disease", "Tissue Site", "Tumor Type", "CELL_LINE_ID", "Growth Medium", "Growth Pattern", "ECM Bio Coatings", "Root Material Type"]]
Premium_data = Premium_data.replace(r'^\s*$', np.nan, regex=True)
Premium_data = Premium_data.rename(columns={"PT_ID": "PT-ID", "Passage Sample": "SM-ID", "JIRA Status": "Status", "Primary disease": "Primary Disease", "CELL_LINE_ID":"Cell Line", "Growth Medium":"Media type", "Growth Pattern":"Dimension", "ECM Bio Coatings":"Flask coating", "Root Material Type":"Starting Material Type"})
Premium_data["Fail Mode"] = 'Not Reported'
Premium_data["Dataset"] = "Premium"

/opt/miniconda3/envs/Broad_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5,33,80,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [1057]:
Premium_data_meta

,Assayed Sample,Assay Date,Assay Level,Assay Result,Assay Type,Basal Medium,Batch Created,Batch ID,Batch Last Updated,Batch Min Passage,...,% FBS,Primary Disease RAW,PT_ID,Receipt Date,Root Material Type,Root Sample Barcode,Sample Type,TC-treated Surface,Tissue Site,Tumor Type
0,NaN,NaN,NaN,NaN,NaN,NaN,2/2/2018 10:35:14 AM,BP-94398,1/8/2019 9:53:20 AM,0.0,...,NaN,Lung cancer,PT-1GMQ3,12/12/2014 11:40:33 AM,"Cells:Cell Line, Viable",SM-6ER9E,Tumor,TC-treated polystyrene,Pleural Cavity,Metastatic
1,NaN,NaN,NaN,NaN,NaN,NaN,2/2/2018 10:35:14 AM,BP-94398,1/8/2019 9:53:20 AM,0.0,...,NaN,Lung cancer,PT-1GMQ3,12/12/2014 11:40:33 AM,"Cells:Cell Line, Viable",SM-6ER9E,Tumor,TC-treated polystyrene,Pleural Cavity,Metastatic
2,NaN,NaN,NaN,NaN,NaN,NaN,2/2/2018 10:35:14 AM,BP-94398,1/8/2019 9:53:20 AM,0.0,...,NaN,Lung cancer,PT-1GMQ3,12/12/2014 11:40:33 AM,"Cells:Cell Line, Viable",SM-6ER9E,Tumor,TC-treated polystyrene,Pleural Cavity,Metastatic
3,SM-HB43Z,5/16/2018 12:00:00 AM,1st Verification,Pass,In-line genomic QC,NaN,2/2/2018 10:35:14 AM,BP-94398,1/8/2019 9:53:20 AM,0.0,...,NaN,Lung cancer,PT-1GMQ3,12/12/2014 11:40:33 AM,"Cells:Cell Line, Viable",SM-6ER9E,Tumor,TC-treated polystyrene,Pleural Cavity,Metastatic
4,NaN,NaN,NaN,NaN,NaN,NaN,2/2/2018 10:35:14 AM,BP-94398,1/8/2019 9:53:20 AM,0.0,...,NaN,Lung cancer,PT-1GMQ3,12/12/2014 11:40:33 AM,"Cells:Cell Line, Viable",SM-6ER9E,Tumor,TC-treated polystyrene,Pleural Cavity,Metastatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9296,SM-I6UNM,10/11/2018 12:00:00 AM,1st Verification,Pass,ELISA,NaN,11/2/2018 12:29:21 PM,BP-102837,6/24/2019 10:44:30 AM,0.0,...,NaN,Renal Translocation Carcinoma,PT-2DT8F,7/30/2018 3:21:07 PM,Tissue:Fresh Tissue,SM-HP7W3,Tumor,TC-treated polystyrene,Kidney,Primary
9297,NaN,NaN,NaN,NaN,NaN,"OPAC: OPAC 39.0 %,OPAC: RspondinI CM 10.0 %,O...",9/27/2018 3:17:40 PM,BP-101506,2/1/2019 12:28:07 PM,0.0,...,NaN,mixed adeno-neuroendocrine (MANEC) of the GE j...,PT-2HF7A,9/27/2018 10:26:21 AM,Tissue:Fresh Tissue,SM-HPADL,Tumor,TC-treated polystyrene,Liver,Metastatic
9298,NaN,NaN,NaN,NaN,NaN,"OPAC: OPAC 39.0 %,OPAC: RspondinI CM 10.0 %,O...",9/27/2018 3:17:40 PM,BP-101506,2/1/2019 12:28:07 PM,0.0,...,NaN,mixed adeno-neuroendocrine (MANEC) of the GE j...,PT-2HF7A,9/27/2018 10:26:21 AM,Tissue:Fresh Tissue,SM-HPADL,Tumor,TC-treated polystyrene,Liver,Metastatic
9299,NaN,NaN,NaN,NaN,NaN,"OPAC: OPAC 39.0 %,OPAC: RspondinI CM 10.0 %,O...",9/27/2018 3:17:40 PM,BP-101506,2/1/2019 12:28:07 PM,0.0,...,NaN,mixed adeno-neuroendocrine (MANEC) of the GE j...,PT-2HF7A,9/27/2018 10:26:21 AM,Tissue:Fresh Tissue,SM-HPADL,Tumor,TC-treated polystyrene,Liver,Metastatic


In [909]:
# Merge CLF and Premium
data = pd.concat([CLF_data, Premium_data])

# Remove Normals
data = data[data["T/N"] == "Tumor"]

# Cleanup some of columns
data["SM-ID"] = [re.split('\r|  ',x)[0].strip(' \t\n\r') if str(x) != 'nan' else x for x in list(data["SM-ID"])]
data["Resolution"] = [str(x) for x in list(data["Resolution"])]
data["Cell Line"] = [str(x) for x in list(data["Cell Line"])]

# Note Dropping lot of conditions
Resolution_dict = {
    'Terminated':'Terminated',
    'In Process':'In Process',
    'Verification':'In Process',
    'Ambiguous':'Drop',
    'Completed':'Completed',
    'Verified Normal':'Verified Normal',
    'Verified Tumor':'Verified Tumor',
    'Clinical team follow up':'In Process',
    'Other verified tumor models':'Verified Tumor',
    'Primary sample verified normal':'Verified Normal',
    'Blacklisted; FP Failure':'Drop',
    'Low Purity Tumor':'Drop',
    'Clinical team follow up,Primary sample verified normal':'Verified Normal',
    'Clinical team follow up,Other verified tumor models':'Verified Tumor',
    'nan':'nan',
    'Abandoned':'Drop',
}

data["Resolution_modified"] = data["Resolution"]
data = data.replace({"Resolution_modified": Resolution_dict})
data = data[(data["Resolution_modified"] != "Drop") & (data["Resolution_modified"] != "In Process")]

### 2 Read and combined Panel data

In [925]:
# TWIST Panel data
TWIST_samples_meta = pd.read_csv(data_location / "PANCAN_TWIST_sample.tsv", sep="\t")
TWIST_pair_meta = pd.read_csv(data_location / "PANCAN_TWIST_pair.tsv", sep="\t")
TWIST_samples = TWIST_samples_meta[["entity:sample_id", "external_id_validation", "participant", "patient_id", "sample_type", "renamed_bai_file", "renamed_bam_file", "depth_of_cov_qc_result", "cnv_calls", "media"]]
TWIST_samples['tn_decision_clean'] = "Not Reported"
TWIST_pair = TWIST_pair_meta[["case_sample", "filtered_variants", "mutect1_callstats", "mutect1_coveragewig", "mutect1_powerwig", "mutect1_vcf", "mutect2_vcf", "oncotated_maf_mutect1", "oncotated_maf_mutect2"]]

TWIST_samples = TWIST_samples[TWIST_samples["entity:sample_id"].isin(TWIST_pair["case_sample"])]
TWIST_samples = TWIST_samples.rename(columns={"entity:sample_id":"case_sample"})
TWIST = pd.merge(TWIST_samples, TWIST_pair, on='case_sample')
TWIST = TWIST.replace(r'^\s*$', np.nan, regex=True)
TWIST["Dataset"] = "TWIST"

<ipython-input-925-dde66bea5383>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TWIST_samples['tn_decision_clean'] = "Not Reported"


In [926]:
# TSCA Panel data
TSCA_samples_meta = pd.read_csv(data_location / "TSCA_HCMI_sample.tsv", sep="\t")
TSCA_pair_meta = pd.read_csv(data_location / "TSCA_HCMI_pair.tsv", sep="\t")
TSCA_samples = TSCA_samples_meta[["entity:sample_id", "bsp_sample_id_validation", "external_id_validation", "participant", "sample_type", "renamed_bai_file", "renamed_bam_file", "depth_of_cov_qc_result", "cnv_calls", "media", "tn_decision_clean"]]
TSCA_pair = TSCA_pair_meta[["case_sample", "filtered_variants", "mutect1_callstats", "mutect1_coveragewig", "mutect1_powerwig", "mutect1_vcf", "mutect2_vcf", "oncotated_maf_mutect1", "oncotated_maf_mutect2"]]
TSCA_pair = TSCA_pair.dropna()

TSCA_samples = TSCA_samples[TSCA_samples["entity:sample_id"].isin(TSCA_pair["case_sample"])]
TSCA_samples = TSCA_samples.rename(columns={"entity:sample_id":"case_sample"})
TSCA = pd.merge(TSCA_samples, TSCA_pair, on='case_sample')
TSCA = TSCA.drop(columns=["case_sample"])
TSCA = TSCA.rename(columns={"bsp_sample_id_validation":"case_sample"})
TSCA = TSCA.replace(r'^\s*$', np.nan, regex=True)
TSCA["Dataset"] = "TSCA"

In [936]:
# Merge TWIST and TSCA with Tableau_data
Panel = pd.concat([TWIST, TSCA])

#Adding PANEL column head
Panel = Panel.rename(columns={"patient_id":"PT-ID"})
Panel.columns = ["PANEL_" + x if i > 4 else x for i,x in enumerate(Panel.columns)]

Panel["participant"] = [str(x) for x in Panel["participant"]]
Panel["PANEL_tn_decision_clean"] = [str(x) for x in Panel["PANEL_tn_decision_clean"]]

### 3 Read RNA data

In [847]:
# RNA data
RNA_meta = pd.read_csv(data_location / "CCLF_RNA_2_0.tsv", sep="\t")
RNA = RNA_meta[["entity:sample_id", 
                     "bsp_sample_id_rna",  
                     "participant", 
                     "sample_type", 
                     "fastq1", 
                     "fastq2", 
                     "bai_file_rna", 
                     "bam_file_rna", 
                     "fusion_predictions", 
                     "fusion_predictions_abridged", 
                     "rnaseqc_count_metrics", 
                     "rnaseqc_count_outputs", 
                     "rnaseqc_exon_counts", 
                     "rnaseqc_gene_counts", 
                     "rnaseqc_gene_rpkm", 
                     "rsem_genes", 
                     "rsem_isoforms", 
                     "star_bam_file", 
                     "star_bam_index", 
                     "star_chimeric_bam_index", 
                     "star_chimeric_junctions", 
                     "star_junctions", 
                     "star_junctions_pass1", 
                     "star_md_bam_file", 
                     "star_md_bam_index", 
                     "star_read_counts",
                    "star_transcriptome_bam"]]

In [848]:
#Adding RNA column head
RNA.columns = ["RNA_" + x if i > 3 else x for i,x in enumerate(RNA.columns)]

In [873]:
# Removing without SM-ID
total_rows = len(RNA)
RNA["bsp_sample_id_rna"] = [str(x) for x in RNA["bsp_sample_id_rna"]]
RNA["participant"] = [str(x) for x in RNA["participant"]]

### 4 Read WES data

In [177]:
# WES data
WES_samples_meta = pd.read_csv(data_location / "CCLF_WES_sample.tsv", sep="\t")
WES_pair_meta = pd.read_csv(data_location / "CCLF_WES_pair.tsv", sep="\t")
WES_samples = WES_samples_meta[["entity:sample_id", "bsp_sample_id_capture", "external_id_capture", "participant", "sample_type", "formatted_bai_file", "formatted_bam_file", "cnv_calls"]]

# Pair is wierd

### 5 Loop through data samples

In [1059]:
def growth_check(x):
    # Checking resolution and mapping back
    x["Fail Mode"] = [str(x) for x in x["Fail Mode"]]
    
    mapping_dict = {
        'Terminated No Growth' : 0,
        'Terminated Unlisted': 0,
        'Terminated Fungus Contamination': 0,
        'Terminated Stopped Growing': 0,
        'Terminated Fungus Contamination,No Growth': 0,
        'Terminated Bacterial Contamination': 0,
        'Terminated Bacterial Contamination,No Growth': 0,
        'Terminated Bacterial Contamination,Fungus Contamination,Sample Quality': 0,
        'Terminated Fibroblast Overgrowth': 0,
        'Terminated Feeder Contamination,No Growth': 0,
        'Verified Normal Fibroblast Overgrowth': 0,
        'Verified Tumor Stopped Growing': 0,
        'Terminated Feeder Contamination': 0,
        'Verified Tumor Fingerprinting Failure': 0,
        'Verified Tumor Unlisted': 0,
        'Verified Normal Mouse Contamination': 0,
        'Verified Normal Stopped Growing': 0,
        'Verified Normal Unlisted': 0,
        'Verified Normal Sample Purity': 0,
        'Terminated Fingerprinting Failure': 0,
        'Terminated Bacterial Contamination,Fungus Contamination': 0,
        'Terminated Bacterial Contamination,Fungus Contamination,No Growth': 0,
        'Terminated No Growth,Sample Quality': 0,
        'Terminated Sample Quality': 0,
        'Terminated Lab Error,Sample Purity': 0,
        'Terminated Mouse Contamination': 0,
        'Terminated Fibroblast Overgrowth,Mouse Contamination': 0,
        'Verified Tumor No Growth': 0,
        'Terminated No Growth After Thaw': 0,
        'Terminated No Growth,Sample Purity,Sample Quality': 0,
        'Terminated Lab Error': 0,
        'Terminated Fingerprinting Failure,Lab Error': 0,
        'Verified Normal No Growth': 0,
        'Terminated No Growth,Sample Purity': 0,
        'Terminated Sample Purity': 0,
        'Verified Tumor Not Reported': 1,
        'Completed Not Reported': 1,
        'nan Not Reported': 'nan',
        'Verified Normal Not Reported': 0,
        'nan': 'nan',
    }
    
    series = x["Resolution_modified"] + " " + x["Fail Mode"]
    series = [str(x) for x in series]
    return series

In [1061]:
series = growth_check(data)

In [1062]:
Counter(series)

Counter({'Terminated No Growth': 1441,
         'Completed nan': 106,
         'Terminated Unlisted': 94,
         'Terminated Fungus Contamination': 74,
         'Terminated Stopped Growing': 713,
         'Terminated Fungus Contamination,No Growth': 2,
         'Terminated Bacterial Contamination': 67,
         'Terminated Bacterial Contamination,No Growth': 20,
         'Terminated Bacterial Contamination,Fungus Contamination,Sample Quality': 3,
         'Terminated Fibroblast Overgrowth': 20,
         'Terminated Feeder Contamination,No Growth': 1,
         'Verified Normal nan': 451,
         'Verified Normal Fibroblast Overgrowth': 2,
         'Verified Tumor nan': 419,
         'Verified Tumor Stopped Growing': 12,
         'Terminated Feeder Contamination': 1,
         'Verified Tumor Fingerprinting Failure': 1,
         'Verified Tumor Unlisted': 45,
         'Verified Normal Mouse Contamination': 3,
         'Verified Normal Stopped Growing': 9,
         'Verified Normal Unli

In [1013]:
spreadsheet = []
empty_numpy_panel = np.empty([1,len(list(Panel)[5:])])
empty_numpy_panel.fill(np.nan)
empty_frame_panel = pd.DataFrame(empty_numpy_panel, columns=list(Panel)[5:])

for i, cell_line in enumerate(data["Cell Line"].unique()):
    data_tmp = data[data["Cell Line"] == cell_line]
    # At least one that isn't Verified Normal
    if all(data_tmp["Resolution_modified"] == "Verified Normal") != True:
        # First get the right data from Panel/RNA
        if cell_line.endswith("T"):
            Panel_tmp = Panel[Panel["participant"] == cell_line[:-1]]
            RNA_tmp = RNA[RNA["participant"] == cell_line[:-1]]
        else:
            Panel_tmp = Panel[Panel["participant"] == cell_line]
            RNA_tmp = RNA[RNA["participant"] == cell_line]

        #Add Panel data by looping through
        if not Panel_tmp.empty:
            # Check if all are not VN
            if all(Panel_tmp["PANEL_tn_decision_clean"] == "VN") != True:
                # Check if multiple sequencing results for cell line
                if len(Panel_tmp) == 1:
                    print('yes')
                    
                    break
                else:
                    break
            #else:
                #Add empty columns
        #else:
            #Add empty columns

yes


In [1014]:
Panel_tmp

,case_sample,external_id_validation,participant,PT-ID,sample_type,PANEL_renamed_bai_file,PANEL_renamed_bam_file,PANEL_depth_of_cov_qc_result,PANEL_cnv_calls,PANEL_media,PANEL_tn_decision_clean,PANEL_filtered_variants,PANEL_mutect1_callstats,PANEL_mutect1_coveragewig,PANEL_mutect1_powerwig,PANEL_mutect1_vcf,PANEL_mutect2_vcf,PANEL_oncotated_maf_mutect1,PANEL_oncotated_maf_mutect2,PANEL_Dataset
1828,SM-D59IN,HG020T_primary,HG020,NaN,Tumor,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/e...,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/e...,pass,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/9...,primary,VT,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/8...,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/9...,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/9...,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/9...,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/9...,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/9...,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/9...,gs://fc-c23078b3-05b3-4158-ba8f-2b1eeb1bfa16/9...,TSCA


In [1032]:
data["Resolution_modified"] + " " + data["Fail Mode"]

0               Terminated No Growth
330                              NaN
331                              NaN
332                              NaN
333                              NaN
                    ...             
9274    Verified Normal Not Reported
9275         Terminated Not Reported
9278                nan Not Reported
9295                nan Not Reported
9296                nan Not Reported
Length: 12724, dtype: object

In [948]:
Panel["PANEL_tn_decision_clean"].unique()

array(['Not Reported', 'VT', 'nan', 'VN'], dtype=object)

In [910]:
Counter(data["Resolution_modified"])

Counter({'Terminated': 6358,
         'Completed': 265,
         'Verified Normal': 2587,
         'Verified Tumor': 2152,
         'nan': 1362})

In [ ]:
# Check if primary
Lookup_table = {"PRIMARY":13,
               "P0":12,
               "P1":11,
               "P2":10,
               "P3":9,
               "P4":8,
               "P5":7,
               "P6":6,
               "P7":5,
               "P8":4,
               "P9":3,
               "P10":2,
               "P11":1,
               "P12":0,}